In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sc
import xrft as xf

In [ ]:
# !pip install xrft

In [ ]:
theta = 1.8113*np.pi/180
N2 = 1*10**(-5)
delta = 0.5
Vinf = 0.1
f = 1e-4
S2 = N2*np.tan(theta)**2/f**2
gamma = 0.5002284142579378 #((np.cos(theta)*(1+S2*(1-delta)))**(-1)+(3-S2)*(np.cos(theta)*(3*(1+S2)-delta*4*S2))**(-1))/2
h = (f*Vinf)/(N2*gamma*np.tan(theta))
lmbd = (N2*gamma*np.tan(theta))/f
fstar = np.cos(theta)*(f**2+N2*np.tan(theta)**2)**(0.5)
beta = np.cos(theta)*(1+S2)**(0.5)

In [ ]:
# file = "olddata/flow_fields_height_103.0_theta_0.005_stratification_1.0e-5_interior_velocity_0.05.nc"
# file = "new_data/paper_data/flow_fields_height_109.0_theta_0.01_stratification_1.0e-5_interior_velocity_0.1_delta_0.1_bo_0_visc_5.0e-5_test.nc"
# TKE_terms_height_98.81324554807192_theta_0.29_stratification_2.0e-7_interior_velocity_0.001_visc_1.0e-6_Sinf_0.0005123752508559594_gamma_0.999713981880998_f_0.0001.nc

In [ ]:
# file = "olddata/dataflow_fields_height_103.0_theta_0.005_stratification_1.0e-5_interior_velocity_0.05.nc"
file = "flow_fields_height_63.21485960107034_interior_velocity_0.1_visc_1.0e-5_Sinf_1.0000287024749517_gamma_0.5002284142579378_theta_1.8113_f_0.0001_N2_1.0e-5.nc"
ocean_data=xr.open_dataset("/glade/derecho/scratch/knudsenl/data/new_data/vinffixed/"+file)#.isel(time=slice(0,4000)) #i=103
seconds = ocean_data.time/pd.Timedelta("1s")
ocean_data = ocean_data.assign_coords(time = ('time',seconds.data))

In [ ]:
def z_change(t,z,x):
    # mean = (ocean_data.B.isel(zC=z, xC=x)).mean()/(N2*np.sin(theta))
    # return ocean_data.B.sel(time=t, method="nearest").isel(zC=z, xC=x)/(N2*np.sin(theta)) - mean
    z_osc = f*np.cos(theta)*delta*lmbd/(fstar**2)*(np.cos(fstar*t)-1)*(h-z)
    # x_mean = N2*np.sin(theta)
    # data = (ocean_data.B+ocean_data.ba).sel(time=t,zC=z, xC=x,method="nearest")
    # z_osc = ocean_data-ocean_data.mean("time")
    # x_mean =(ocean_data.B+ocean_data.ba).sel(time=t,zC=z,method="nearest").differentiate('xC').mean()
    return z_osc #/x_mean

In [ ]:
# ocean_data.B.isel(xC=256,yC=0,zC=32).plot(color="blue")
# ocean_data.vb.isel(xC=256,yF=0,zC=32).plot(color="red")

In [ ]:
# ti = 100
# tf = 375
# lagrangian_z = z_change(ocean_data.time[ti:tf],ocean_data.zC[32],ocean_data.xC[256])
# plt.plot(lagrangian_z)

In [ ]:
import matplotlib.colors as colors

In [ ]:
inertial_period = ocean_data.time*fstar/(2*np.pi)
ocean_data_ip = ocean_data.assign_coords(intp = ('time',inertial_period.data))

In [ ]:
inertial_period[250]

In [ ]:
ocean_data_ip.intp[250]

In [ ]:
# ocean_data_ip.u.isel(time=tau,zC=slice(0,64))*ocean_data_ip.b.isel(time=tau,zC=slice(0,64))

In [ ]:
tau = 140
lagrangian_z = z_change(ocean_data.time[tau],ocean_data.zC[40],ocean_data.xC[230])
ocean_data_ip.u.isel(time=tau,zC=slice(0,140)).plot(norm=colors.SymLogNorm(linthresh=1e-12))  # norm=colors.SymLogNorm(linthresh=1e-10)
(ocean_data_ip.B).isel(time=tau,yC=0,zC=slice(0,140)).plot.contour()
# plt.scatter(ocean_data.xC[230]+lagrangian_z,ocean_data.zC[40], color="yellow")
# lagrangian_z

In [ ]:
(ocean_data.B.isel(zC=32,xC=256,yC=0)-ocean_data.B.isel(zC=32,xC=256,yC=0).mean()).plot()

In [ ]:
ocean_data.xC[409]

In [ ]:
# ocean_data.Ri.sel(xC=ocean_data.xC[xi],zF=ocean_data.zC[zi],time=ocean_data.time[ti:tf],method="nearest").plot()

In [ ]:
ocean_data.zC[40]

In [ ]:
# z_change(ocean_data.time[ti:tf],zi,xi)

In [ ]:
ti = 200
tf = 800
zi = 40
xi = 512
v_lagrange = ocean_data.v.sel(xC=ocean_data.xC[xi]-z_change(ocean_data.time[ti:tf],ocean_data.zC[zi],ocean_data.xC[xi]),zC=ocean_data.zC[zi],time=ocean_data.time[ti:tf],method="nearest")

In [ ]:
ti = 200
tf = 800
zi = 40
xi = 256
u_lagrange = ocean_data.u.sel(xF=ocean_data.xF[xi]-z_change(ocean_data.time[ti:tf],ocean_data.zC[zi],ocean_data.xF[xi]),zC=ocean_data.zC[zi],time=ocean_data.time[ti:tf],method="nearest")

In [ ]:
u_lagrange.time[-1]*(fstar)/(2*np.pi)

In [ ]:
u_lagrange = u_lagrange.drop_vars(['xF','yC','zC'])
v_lagrange = v_lagrange.drop_vars(['xC','yF','zC'])

In [ ]:
plt.plot(u_lagrange.time*(fstar)/(2*np.pi),u_lagrange.values/u_lagrange.values[0],color = "r",label=f"$u/u_2$ at z={ocean_data_ip.zC[zi].values}")
plt.plot(v_lagrange.time*(fstar)/(2*np.pi),v_lagrange.values/u_lagrange.values[0],color = "k",alpha=0.8,label=f"$v/u_2$ at z={ocean_data_ip.zC[zi].values}")
# plt.xlim([4.8,15.2])
plt.xlabel("Interial Period $(tf^*(2\pi)^{-1})$")
plt.yscale("symlog",linthresh=1)
plt.legend(loc="lower left")
plt.text(8,1000,"(a.)", ha="left",va="top")
plt.savefig("S1_gamma_l_delta_0_5_quasi_lagrangian_time_series.pdf")
# plt.xticks([5,7,9,11,13,15])
plt.show()

In [ ]:
plt.plot(u_lagrange.time[0:81]*(fstar)/(2*np.pi),u_lagrange.values[0:81]/v_lagrange.values[0],color = "r",label="$u/v_{10}$ at z="+str(ocean_data_ip.zC[20].values))
plt.plot(v_lagrange.time[0:81]*(fstar)/(2*np.pi),v_lagrange.values[0:81]/v_lagrange.values[0],color = "k",alpha=0.8,label="$u/v_{10}$ at z="+str(ocean_data_ip.zC[20].values))
# plt.xlim([10,14])
# plt.ylim([-18,18])
# plt.xlabel("Interial Period $(tf^*(2\pi)^{-1})$")
# plt.yscale("symlog",linthresh=1)
plt.legend(loc="lower left",prop={'size':9})
# plt.gca().set_aspect(1.5)
plt.xticks([])
# plt.text(9.4,11.5,"(b.)", ha="left",va="top")
# plt.figure(figsize=(12,8))
# plt.savefig("quasi_lagrangian_time_series_10_14.png")

plt.show()

In [ ]:
ti = 200
tf = 281
zi = 20
xi = 256
plt.plot(ocean_data.time[ti:tf]*(fstar)/(2*np.pi),ocean_data.ub.sel(xF=ocean_data.xC[xi],zC=ocean_data.zC[zi],time=ocean_data.time[ti:tf],method="nearest").values/ocean_data.vb.sel(xC=ocean_data.xC[xi],zC=ocean_data.zC[zi],time=ocean_data.time[ti],method="nearest").values,color = "r",label="$U/V_{10}$ at z="+str(ocean_data_ip.zC[zi].values))
plt.plot(ocean_data.time[ti:tf]*(fstar)/(2*np.pi),ocean_data.vb.sel(xC=ocean_data.xC[xi],zC=ocean_data.zC[zi],time=ocean_data.time[ti:tf],method="nearest").values/ocean_data.vb.sel(xC=ocean_data.xC[xi],zC=ocean_data.zC[zi],time=ocean_data.time[ti],method="nearest").values,color = "k",alpha=0.8,label="$V/V_{10}$ at z="+str(ocean_data_ip.zC[zi].values))
plt.xlim([10,14])
plt.xlabel("Interial Period $(tf^*(2\pi)^{-1})$")
# plt.yscale("symlog",linthresh=1)
plt.legend(loc="lower left",prop={'size':9})
# plt.gca().set_aspect(1.1)
# plt.figure(figsize=(12,8))
plt.text(9.4,1,"(c.)", ha="left",va="top")
plt.savefig("background_time_series_10_14.png")
# plt.xticks([5,7,9,11,13,15])
plt.show()

In [ ]:
transform_lagrange = xf.power_spectrum(u_lagrange,dim="time")

In [ ]:
np.abs(transform_lagrange[1601:]).values[:,0].argsort()[-4:]

In [ ]:
# (transform_lagrange[1601:].freq_time[90]+transform_lagrange[1601:].freq_time[68])*2*np.pi

In [ ]:
# transform_lagrange[1601:].freq_time[67]

In [ ]:
transform_lagrange.plot()
plt.xscale("symlog",linthresh=1e-10)

In [ ]:
fig, ax_dict = plt.subplot_mosaic([['(a.)','(b.)']],layout="constrained",figsize=(9,4))
ax = ax_dict['(b.)']
ax.plot(u_lagrange.time[:201]*(fstar)/(2*np.pi),u_lagrange.values[:201]/u_lagrange.values[0],color = "r",label="$u/u_{10}$ at z=16.02 m")
ax.plot(v_lagrange.time[:201]*(fstar)/(2*np.pi),v_lagrange.values[:201]/u_lagrange.values[0],color = "k",alpha=0.8,label="$v/u_{10}$ at z=16.02 m")
# plt.xlim([4.8,15.2])
ax.set_xlabel("Modified Inertial Period")
ax.set_xlim([10,20])
ax.set_yscale("symlog",linthresh=1)
ax.legend(loc="lower left")
ax.set_box_aspect(0.75)
ax.set_title("(b.)", fontfamily='serif', loc='left', fontsize='medium')
ax = ax_dict['(a.)']
ax.plot(transform_lagrange.freq_time[:]*2*np.pi,transform_lagrange[:],color="k")
ax.vlines(0.5*fstar,ymin=0,ymax=np.max(transform_lagrange)*1.05,color="b",alpha=0.5,linestyles="dotted",label="$f^*/2$")
ax.vlines(fstar,ymin=0,ymax=np.max(transform_lagrange)*1.05,color="k",alpha=0.5,linestyles="dotted",label="$f^*$")
ax.set_xlim([1e-05,transform_lagrange.freq_time[:].max()*2*np.pi])
ax.set_ylim([0,np.max(transform_lagrange)*1.05])
ax.set_box_aspect(0.9)
ax.set_title("(a.)", fontfamily='serif', loc='left', fontsize='medium')
ax.legend()
ax.set_xlabel("Frequency [s$^{-1}$]")
ax.set_xscale("log")
ax.set_ylabel("Power Spectrum Density [m$^3$ s$^{-2}$]")
plt.savefig("S_1_gamma_l_delta_0_5_power_spectral_density_times_series.pdf")
plt.show()

In [ ]:
plt.plot(transform_lagrange.freq_time[:]*2*np.pi,transform_lagrange[:],color="k")
plt.vlines(0.5*fstar,ymin=0,ymax=np.max(np.abs(transform_lagrange))*1.1,color="b",alpha=0.5,linestyles="dotted",label="$f^*/2$")
plt.vlines(fstar,ymin=0,ymax=np.max(np.abs(transform_lagrange))*1.1,color="k",alpha=0.5,linestyles="dotted",label="$f^*$")
plt.xlim([1.1218508e-05,0.001])
# plt.ylim([0.1,30])
plt.legend()
plt.xlabel("Frequency [s$^{-1}$]")
plt.xscale("log")
# plt.yscale("log")
plt.ylabel("Power Spectrum Density [m$^3$ s$^{-2}$]")
# plt.text(-0.0001,0.23,"(b.)", ha="left",va="top")
# plt.title("Quasi-Lagrangian Fourier Transform of u-velocity")
plt.savefig("QL_fourier_transform.png")
plt.show()

In [ ]:
ti = -200
tf = -1
plt.plot(ocean_data_ip.intp.isel(time=slice(ti, tf)), ocean_data_ip.u.isel(zC=64, yC=0, xF=128, time=slice(ti, tf))/ocean_data_ip.v.isel(zC=64, yF=0, xC=128, time=ti),color = "r",label=f"$u/v_5$ at z={ocean_data_ip.zC[64].values}")
plt.plot(ocean_data_ip.intp.isel(time=slice(ti, tf)), ocean_data_ip.v.isel(zC=64, yF=0, xC=128, time=slice(ti, tf))/ocean_data_ip.v.isel(zC=64, yF=0, xC=128, time=ti),color = "k",label=f"$v/v_5$ at z={ocean_data_ip.zC[64].values}")
# plt.xlim([4.8,15.2])
plt.xlabel("Interial Period $(tf^*(2\pi)^{-1})$")
plt.legend()
# plt.xticks([5,7,9,11,13,15])
plt.show()

In [ ]:
ti = 99
tf = 297
# plt.plot(ocean_data_ip.intp.isel(time=slice(ti, tf)), ocean_data_ip.ub.isel(zC=64, yC=0, xF=128, time=slice(ti, tf))/ocean_data_ip.vb.isel(zC=64, yC=0, xF=128, time=ti),color = "r",label=f"$u/v_5$ at z={ocean_data_ip.zC[64].values}")
plt.plot(ocean_data_ip.intp.isel(time=slice(ti, tf)), ocean_data_ip.vb.isel(zC=64, yF=0, xC=128, time=slice(ti, tf))/ocean_data_ip.vb.isel(zC=64, yF=0, xC=128, time=ti),color = "k",label=f"$V(t)/V_5$ at z={ocean_data_ip.zC[64].values}")
plt.xlim([4.8,15.2])
plt.xlabel("Interial Period $(tf^*(2\pi)^{-1})$")
plt.ylabel("$V/V_5$")
# plt.legend(loc="upper left")
plt.xticks([5,7,9,11,13,15])
plt.show()

In [ ]:
plt.plot(ocean_data_ip.intp.isel(time=slice(100, 420)), ocean_data_ip.v.isel(zC=63, yF=0, xC=128, time=slice(100, 420)))
# plt.plot(ocean_data_ip.intp.isel(time=slice(100,400)),ocean_data_ip.u.isel(zC=63,yC=0,xC=128,time=slice(100,400))/ocean_data_ip.v.isel(zC=63,yC=0,xC=128,time=100))
# plt.plot(ocean_data_ip.intp.isel(time=slice(100,400)),ocean_data_ip.w.isel(zC=63,yC=0,xC=128,time=slice(100,400))/ocean_data_ip.v.isel(zC=63,yC=0,xC=128,time=100))
# plt.yscale("symlog",linthresh=0.01)

In [ ]:
plt.plot(ocean_data_ip.intp.isel(time=slice(200,400)),ocean_data_ip.va.isel(zC=63,yF=0,xC=256,time=slice(200,400)))
plt.yscale("symlog",linthresh=0.0001)

In [ ]:
plt.plot(ocean_data_ip.intp.isel(time=slice(200,400)),ocean_data_ip.vb.isel(zC=63,yC=0,xF=256,time=slice(200,400)))
plt.yscale("symlog",linthresh=0.0001)

In [ ]:
total = (ocean_data_ip.vb.interp_like(ocean_data_ip.va.xC).interp_like(ocean_data_ip.va.yF)).isel(zC=63,yC=0,xF=256)+ ocean_data_ip.va.isel(zC=63,yF=0,xC=256)

In [ ]:
plt.plot(ocean_data_ip.intp.isel(time=slice(200,400)),total.isel(time=slice(200,400)))
plt.yscale("symlog",linthresh=0.0001)

In [ ]:
zi = 64
ti = 800
tf = -1
# v_mean = ocean_data.v.mean(["xC"])
fourier_data = ocean_data.v.isel(yF=0,zC=zi,xC=256,time=slice(ti,tf))/ocean_data.v.isel(yF=0,zC=zi, xC=256,time=ti).max()
# time_data = ocean_data.intp.isel(time=slice(ti,tf))

In [ ]:
# fourier_data["time"] = fourier_data.time * f/(2*np.pi)
fourier_data.plot()

In [ ]:
# fourier_data_ni = fourier_data.drop(['intp'])
transform = xf.fft(fourier_data,dim="time")

In [ ]:
plt.plot(transform.freq_time*2*np.pi,np.abs(transform)*1e-6,color="k")
plt.vlines(0.5*beta*f,ymin=0,ymax=np.max(np.abs(transform))*1.1*1e-6,color="b",alpha=0.5,linestyles="dotted",label="$\omega=f^*/2$")
plt.xlim([0,0.001])
# plt.ylim([0,3.1])
# plt.legend()
plt.xlabel("Frequency (in $s^{-1})$")
plt.ylabel("Absolue value of amplitude ($|\chi(\omega)| \cdot 10^{-6}$)")

In [ ]:
for j in [*range(0,65,16)]:
    fourier_data = ocean_data.v.isel(yC=0,zC=j,xC=256,time=slice(ti,tf))/ocean_data.v.isel(yC=0,zC=64, xC=256,time=ti).max()
    transform = xf.fft(fourier_data,dim="time")
    plt.plot(transform.freq_time*2*np.pi,np.abs(transform)*1e-6,label=f"Level at z={ocean_data.zC[j].values}")
# plt.vlines(0.5*beta*f,ymin=0,ymax=5,color="b",alpha=0.5,linestyles="dotted")
# plt.vlines(beta*f,ymin=0,ymax=5,color="b",alpha=0.5,linestyles="dotted")
# plt.vlines(1.5*beta*f,ymin=0,ymax=5,color="b",alpha=0.5,linestyles="dotted")
# plt.vlines(2.5*beta*f,ymin=0,ymax=5,color="b",alpha=0.5,linestyles="dotted")
# plt.legend(loc="upper right")
# plt.yscale("log")
# plt.gca().set_aspect(100)
plt.xlim([0,0.001])
plt.xticks(rotation=15)
# plt.xscale("symlog",linthresh=10**(-3))
plt.show()

In [ ]:
magnitude = np.abs(fourier_data)
growth_rate = sc.stats.linregress(fourier_data.time, np.log(np.abs(fourier_data/fourier_data[0])))

In [ ]:
plt.plot(time_data*fstar/(2*np.pi),magnitude)
# plt.yscale("log")

In [ ]:
growth_rate.slope

In [ ]:
normalized_fourier_data = fourier_data/np.exp(growth_rate.slope*fourier_data.time)

In [ ]:
transform

In [ ]:
# fourier_data_ni = fourier_data.drop(['intp'])
transform = xf.fft(normalized_fourier_data)
np.abs(transform).plot()
# plt.vlines(0.5*fstar,ymin=0,ymax=np.max(transform.imag)*1.1,color="b",alpha=0.5,linestyles="dotted")

In [ ]:
plt.plot(time_data,normalized_fourier_data/np.abs(normalized_fourier_data[0]))
plt.xlabel("time ($t\dfrac{f^*}{2\pi}$)")
plt.ylabel("normalized velocity ($m s^{-1}$)")
# plt.yscale("log")

In [ ]:
sr = len(time_data)/(time_data[-1]-time_data[0]).values
ts = (time_data[-1]-time_data[0])/sr
ft = np.fft.rfft(normalized_fourier_data,norm=None)
N = len(ft)
n = np.arange(N)
T = (normalized_fourier_data.time[-1]-normalized_fourier_data.time[-2]).values
freq = np.fft.rfftfreq(len(normalized_fourier_data),d=0.05)
plt.plot(freq,np.abs(ft),color="k")
# plt.vlines(0.5*fstar*10**(4),ymin=0,ymax=np.max(np.abs(ft))*1.1,color="b",alpha=0.5,linestyles="dotted")
# plt.vlines(1.5*fstar*10**(4),ymin=0,ymax=np.max(np.abs(ft))*1.1,color="b",alpha=0.5,linestyles="dotted")
# plt.vlines(2.5*fstar*10**(4),ymin=0,ymax=np.max(np.abs(ft))*1.1,color="b",alpha=0.5,linestyles="dotted")
# plt.vlines(3.5*fstar*10**(4),ymin=0,ymax=np.max(np.abs(ft))*1.1,color="b",alpha=0.5,linestyles="dotted")
# plt.vlines(4.5*fstar*10**(4),ymin=0,ymax=np.max(np.abs(ft))*1.1,color="b",alpha=0.5,linestyles="dotted")
# plt.vlines(5.5*fstar*10**(4),ymin=0,ymax=np.max(np.abs(ft))*1.1,color="b",alpha=0.5,linestyles="dotted")
# plt.vlines(6.5*fstar*10**(4),ymin=0,ymax=np.max(np.abs(ft))*1.1,color="b",alpha=0.5,linestyles="dotted")

# plt.vlines(6.5*fstar*10**(4),ymin=0,ymax=0.0007)
# plt.xlim([0,0.0001])
plt.xlabel("Frequency (per intertial period)")
plt.ylabel("FFT Amplitude")

In [ ]:
# v_mean = ocean_data.v.mean(["xC"])
fourier_data_map = (ocean_data.v.isel(time=slice(200,400))/ocean_data.v.isel(time=200).max())

In [ ]:
fourier_data_map = fourier_data_map.mean("yC")
fourier_data_map

In [ ]:
transform_domain = xf.fft(fourier_data_map,dim=["zC"])

In [ ]:
plt.plot(transform_domain.freq_zC,transform_domain.isel(time=2,xC=256).real)

In [ ]:
xf.ifft(transform_domain)

In [ ]:
plt.plot(transform_domain.freq_xC,transform_domain.real[63,:,:])

In [ ]:
import matplotlib.colors as colors

In [ ]:
# plt.contourf(transform_domain.freq_xC,transform_domain.freq_time*2*np.pi,transform_domain.real[:,63,:],cmap='Accent',norm=colors.SymLogNorm(linthresh=1e9))
# plt.colorbar()
# plt.hlines(fstar,xmin=-0.1,xmax=0.1)